Crear canva

In [16]:
pip install torch torchvision torchaudio

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
    --------------------------------------- 0.0/2.4 MB 435.7 kB/s eta 0:00:06
   --- ------------------------------------ 0.2/2.4 MB 1.8 MB/s eta 0:00:02
   ------------------------ --------------- 1.5/2.4 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 11.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import torch

In [21]:
# Esta celda realiza las siguientes operaciones:
# - Configura un lienzo interactivo para dibujar formas.
# - Implementa funciones para capturar dibujos y procesarlos.

from ipycanvas import Canvas # Proporciona la funcionalidad para crear lienzos interactivos en Jupyter Notebook.
from PIL import Image, ImageDraw # Se utiliza para trabajar con imágenes, como convertir de matrices a imágenes y viceversa.
import numpy as np # Es esencial para realizar operaciones matemáticas en arreglos (como las imágenes que se representan como matrices).
import os # Permite interactuar con el sistema operativo, como crear directorios.

# Se crea un lienzo blanco de 200x200 píxeles.
canvas = Canvas(width=200, height=200, background_color="white", sync_image_data = True)
# La opción sync_image_data = True asegura que los cambios en el lienzo se reflejen en la representación de la imagen.

# Función para capturar el dibujo como imagen
def get_drawing(): # Obtiene los datos de la imagen del lienzo.
    # Convierte la imagen del lienzo a una matriz NumPy y luego a imagen de escala de grises    
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    # Convertir la imagen a escala de grises (L)
    img = img.convert("L")
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    # Convertir la imagen en escala de grises a una matriz NumPy
    return np.array(img)

# Función para guardar el dibujo
def save_drawing(class_name, count): # Crea un directorio para almacenar las imágenes de la clase especificada (por ejemplo, "0", "1", "2" para dígitos).
    # Crear directorio si no existe
    os.makedirs(f"data/{class_name}", exist_ok=True) # exist_ok=True evita errores si el directorio ya existe.
    # Obtiene el dibujo del lienzo utilizando get_drawing().
    img = get_drawing()
    
    # Guardar la imagen en el directorio especificado
    filepath = f"data/{class_name}/{count}.png"
    Image.fromarray(img).save(filepath)
    print(f"Dibujo guardado en: {filepath}")


# Variable para almacenar la última posición
last_x, last_y = None, None

# Función para dibujar en el lienzo
def on_mouse_down(x, y): 
    global last_x, last_y
    canvas.fill_style = "black"
    last_x, last_y = x, y  # Guardar la posición inicial cuando se presiona el botón del mouse

def on_mouse_move(x, y): # Se definen funciones para manejar los eventos de clic, movimiento y liberación del mouse.
    global last_x, last_y
    if last_x is not None and last_y is not None:
        canvas.stroke_style = "black"
        canvas.line_width = 5
        canvas.begin_path()
        canvas.move_to(last_x, last_y)
        canvas.line_to(x, y)
        canvas.stroke()
        last_x, last_y = x, y  # Actualizar la posición de la última coordenada

def on_mouse_up(x, y):
    global last_x, last_y
    last_x, last_y = None, None  # Resetear cuando se suelta el mouse

# Asignar los eventos de mouse al lienzo
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

# Mostrar el lienzo
display(canvas)


Canvas(height=200, sync_image_data=True, width=200)

In [27]:
# Se guarda imagen en un directorio específico.
save_drawing("square", 5)

Dibujo guardado en: data/square/5.png


Generar Imágenes

In [29]:
# Esta celda realiza las siguientes operaciones:
# - Genera datos sintéticos de varias formas geométricas (círculos, cuadrados, etc.).
# - Guarda estas imágenes en una estructura de carpetas.

# Generar formas sintéticas

def generate_synthetic_data(class_name, count): # Esta función genera y guarda imágenes de formas geométricas básicas en blanco y negro.
    os.makedirs(f"data/{class_name}", exist_ok=True)
    for i in range(count):
        # Crear una imagen en blanco
        img = Image.new("L", (28, 28), "white") # Imagen de  28X28 píxeles en escala de grises "L"
        draw = ImageDraw.Draw(img) # Crea un objeto ImageDraw.Draw para dibujar en la imagen.

        # Nombre de la clase de la forma geométrica a generar. Debe ser "circle", "square", "triangle" o "star"
        if class_name == "circle":
            draw.ellipse((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "square":
            draw.rectangle((5, 5, 23, 23), outline="black", fill="black")
        elif class_name == "triangle":
            draw.polygon([(14, 5), (5, 23), (23, 23)], outline="black", fill="black")
        elif class_name == "star":
            draw.polygon([(14, 5), (10, 20), (5, 14), (23, 14), (18, 20)], outline="black", fill="black")

        # Guardar la imagen
        filepath = f"data/{class_name}/{i}.png" # Ruta del archivo 
        img.save(filepath) # Acá se guarda la imagen en la ruta 
        #print(f"Dibujo sintético guardado en: {filepath}")

# Generar 100 imágenes sintéticas por clase
generate_synthetic_data("circle", 100)
generate_synthetic_data("square", 100)
generate_synthetic_data("triangle", 100)
generate_synthetic_data("star", 100)


Cargar imágenes en el Dataset

In [31]:
# Esta celda realiza las siguientes operaciones:
# - Define una clase personalizada para cargar imágenes en un dataset.
# - Aplica transformaciones como normalización y conversión a tensor.

import os
import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class DrawingDataset(Dataset): # root_dir es la ruta del directorio raíz que contiene las imágenes del dataset.
    def __init__(self, root_dir, transform=None): # Metodo que inicializa el dataset, verifica exisitencia de la imagen 
        self.root_dir = root_dir
        self.transform = transform # Del objeto de transforms de torchvision para transformaciones que se aplicarán a las imágenes cargadas.
        self.filepaths = glob.glob(os.path.join(root_dir, "*", "*.png")) # Lista de rutas de las imágenes en el dataset

        if not self.filepaths:
            raise ValueError(f"No se encontraron imágenes en {root_dir}. Verifica la estructura del directorio.")
            
        # Lista de etiquetas correspondientes a cada imagen, basadas en el nombre del directorio padre
        self.labels = [os.path.basename(os.path.dirname(path)) for path in self.filepaths]
        # Diccionario que mapea cada etiqueta única a un índice entero.
        self.label_to_idx = {label: idx for idx, label in enumerate(set(self.labels))}

    # Devuelve el tamaño del dataset (número total de imágenes).
    def __len__(self):
        return len(self.filepaths)

    # Obtiene una imagen y su etiqueta correspondiente a un índice específico
    def __getitem__(self, idx):
            # Abrir imagen y convertirla a escala de grises
            img = Image.open(self.filepaths[idx]).convert("L")  # Convertir a escala de grises (1 canal)
            label = self.label_to_idx[self.labels[idx]] # Obtiene la etiqueta del diccionario
            
            # Aplicar transformaciones
            if self.transform:
                img = self.transform(img)
            else:
                # Si no hay transformaciones, convierte la imagen a un tensor usando
                img = transforms.ToTensor()(img)
            
            return img, label # Devuelve una tupla con la imagen (como tensor) y su etiqueta (como entero).
# Transformaciones para normalizar imágenes
transform = transforms.Compose([
    transforms.Resize((28, 28)),       # Asegurar tamaño 28x28
    transforms.ToTensor(),            # Convertir a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalizar
])

# Cargar el dataset
dataset = DrawingDataset(root_dir="data", transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


Paso 3: Crear y Entrenar el Modelo CNN Definimos un modelo básico en PyTorch.

In [33]:
# Esta celda realiza las siguientes operaciones:
# - Define un modelo básico de PyTorch y entrena una red neuronal CNN.
# - Imprime estadísticas como pérdida y precisión por época.

import torch.nn.functional as F

    # Verificar si GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}") # Imprime el dispositivo utilizado para el entrenamiento.

# Enviar el modelo al dispositivo
model.to(device)

# Función para entrenar el modelo CNN.
# Dataloader: Cargador de datos (DataLoader) creado a partir del dataset DrawingDataset.
# Criterion: Función de pérdida para evaluar el error del modelo (definido más adelante).
# Optimizer: Optimizador para ajustar los pesos del modelo durante el entrenamiento.
# Epochs (opcional, int): Número de épocas para entrenar el modelo (valor por defecto 5).
    
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()  # Modo entrenamiento
    for epoch in range(epochs):
        # Variable para acumular la pérdida total durante una época.
        running_loss = 0.0
        # Variable para acumular el número de predicciones correctas.
        correct = 0
        # total = 0 # Variable para acumular el número total de imágenes en una época.
        total = 0

        # Envía las imágenes y etiquetas al dispositivo seleccionado (GPU o CPU) para el procesamiento.
        for images, labels in dataloader:
            # Enviar imágenes y etiquetas al dispositivo
            images, labels = images.to(device), labels.to(device)

            # Pone a cero los gradientes del optimizador antes de cada paso de retropropagación.
            # Forward
            optimizer.zero_grad()
            outputs = model(images) #  Realiza una pasada hacia adelante por el modelo para obtener las salidas (predicciones)
            loss = criterion(outputs, labels) # Calcula la pérdida entre las salidas del modelo y las etiquetas reales utilizando la función de pérdida.
            
            # Backward y optimización
            loss.backward() # Calcula los gradientes de la pérdida con respecto a los pesos del modelo.
            optimizer.step() # Actualiza los pesos del modelo utilizando el optimizador y los gradientes calculados.
            
            # Estadísticas
            running_loss += loss.item() # Acumula la pérdida del lote actual.
            _, predicted = torch.max(outputs, 1) # Obtiene las clases predichas (índice de la clase con mayor probabilidad).
            total += labels.size(0) # Contabiliza el número de imágenes en el lote actual.
            correct += (predicted == labels).sum().item() # Contabiliza el número de predicciones correctas en el lote actual.

        accuracy = 100 * correct / total #  Calcula la precisión del modelo para la época actual.
        # Imprime el número de la época actual, la pérdida promedio y la precisión para la época.
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader):.4f}, Accuracy: {accuracy:.2f}%")

# Entrenar el modelo
train_model(model, dataloader, criterion, optimizer, epochs=5)


Usando dispositivo: cpu


NameError: name 'model' is not defined

Entrenamos el modelo con los datos capturados.

In [35]:
# Mostrar el lienzo
display(canvas)

Canvas(height=200, image_data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xc8\x00\x00\x00\xc8\x08\x06\x0…

In [37]:
# Esta celda realiza las siguientes operaciones:
# - Utiliza un modelo previamente entrenado para predecir la clase de un dibujo en el lienzo.
# - Limpia el lienzo después de realizar una predicción.

import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# Crear la transformación que se debe aplicar al dibujo
transform = transforms.Compose([
    transforms.ToTensor(),                  # Convierte a tensor
    transforms.Normalize((0.5,), (0.5,))    # Normaliza (ajusta según las necesidades del modelo)
])

# Función para obtener la imagen desde el lienzo
def get_drawing():
    img = Image.fromarray(canvas.get_image_data(0, 0, 200, 200))
    img = img.convert("L")  # Convertir a escala de grises
    img = img.resize((28, 28))  # Redimensionar a 28x28 píxeles
    return np.array(img)  # Retorna como un array NumPy

# Función para predecir usando el modelo
def predict_drawing(model, dataset):
    # Obtener el dibujo actual del lienzo
    img = get_drawing()
    
    # Preprocesar la imagen
    img_tensor = transform(Image.fromarray(img)).unsqueeze(0)  # Agregar dimensión batch
    
    # Enviar la imagen al modelo para predicción
    model.eval()  # Establecer el modelo en modo evaluación
    with torch.no_grad():  # Desactivar gradientes para predicción
        output = model(img_tensor)  # Obtener las predicciones
        pred = torch.argmax(output, dim=1).item()  # Obtener la clase predicha
    
    # Mapear la predicción a la etiqueta correspondiente
    label = list(dataset.label_to_idx.keys())[list(dataset.label_to_idx.values()).index(pred)]
    print(f"Predicción: {label}")

# Ejemplo de uso después de dibujar algo en el lienzo:
predict_drawing(model, dataset)

#Limpiar el canvas después de la prediccion
canvas.clear()



NameError: name 'model' is not defined